In [1]:
import torch
from torch import nn
from torchinfo import summary

from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from tqdm.auto import tqdm
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy  as np
import tracemalloc 

import json
import os
import glob
import pickle
from itertools import combinations
import gc
import time
import random

import warnings
warnings.filterwarnings("ignore")

/home/tran/miniconda3/envs/compgan/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_ROOT = "../data/compgan_dataset/"

train_data_file_name_ = "train_data{}.npy"
train_label_file_name_ = "train_label{}.npy"
test_data_file_name_ = "test_data{}.npy"
test_label_file_name_ = "test_label{}.npy"

TRAIN_FOLDER_PATH = os.path.join(DATA_ROOT, "train")
TEST_FOLDER_PATH = os.path.join(DATA_ROOT, "test")
RESULT_FOLDER_PATH = os.path.join(DATA_ROOT, "results")

assert os.path.isdir(TRAIN_FOLDER_PATH) and os.path.isdir(TEST_FOLDER_PATH)
os.makedirs(RESULT_FOLDER_PATH, exist_ok=True)

data_files = sorted(glob.glob(os.path.join(TRAIN_FOLDER_PATH, train_data_file_name_.format("*"))))
print(data_files)
print(len(data_files))

['../data/compgan_dataset/train/train_data0.npy', '../data/compgan_dataset/train/train_data1.npy', '../data/compgan_dataset/train/train_data10.npy', '../data/compgan_dataset/train/train_data11.npy', '../data/compgan_dataset/train/train_data12.npy', '../data/compgan_dataset/train/train_data13.npy', '../data/compgan_dataset/train/train_data14.npy', '../data/compgan_dataset/train/train_data15.npy', '../data/compgan_dataset/train/train_data2.npy', '../data/compgan_dataset/train/train_data3.npy', '../data/compgan_dataset/train/train_data4.npy', '../data/compgan_dataset/train/train_data5.npy', '../data/compgan_dataset/train/train_data6.npy', '../data/compgan_dataset/train/train_data7.npy', '../data/compgan_dataset/train/train_data8.npy', '../data/compgan_dataset/train/train_data9.npy']
16


In [3]:
USER_NUM = 16
SENSOR_NUM = 7
EACH_SENSOR_CHANNEL = 6
assert USER_NUM == len(data_files)
feature_num = SENSOR_NUM * EACH_SENSOR_CHANNEL

In [4]:
# important

label_list = ['歩行(平地)',
 '歩行(階段)',
 'ベッド上での起き上がり',
 'ベッド椅子間の乗り移り(立つ)',
 'ベッド椅子間の乗り移り(立たない)',
 '立ち座り',
 '座位保持・座位バランス',
 '立位保持・立位バランス',
 '関節可動域増大訓練(肩)',
 '関節可動域増大訓練(股関節)']

label_dict = dict(enumerate(label_list))

In [5]:
# important
eng_label_dict = dict(zip(
    label_list,
    ['Walking', 'Upstair', 'Bed_Standup', 'Change_Bed', 'Change_Bed_Standup', 'Sit_Down', 'Sit', 'Stand', 'Shoulder_Exercise', 'Hip_Exercise']
))

eng_label_list = [eng_label_dict[i] for i in label_list]

In [6]:
class CustomTrainDataset(Dataset):
    TRAIN_MODE = "train"
    TEST_MODE = "test"
    
    def __init__(self, mode, feature_data, label_data, missing_sensor_numbers=0):
        self.mode = mode
        assert mode in [self.TRAIN_MODE, self.TEST_MODE]
        
        self.features = feature_data
        self.label = label_data
        assert len(self.features) == len(self.label), "features len is not equal to label len"
        self.missing_sensor_numbers = missing_sensor_numbers

        self.missing_index_list = []
        for missing_count in range(missing_sensor_numbers + 1):
            for missing_index in combinations(range(SENSOR_NUM), missing_count):
                self.missing_index_list.append(missing_index)

    def transform(self, one_feature, missing_sensor_id_list):
        # Make one sensor data become 0
        one_feature_cp = one_feature.copy()
        
        for missing_sensor_id in missing_sensor_id_list:
            one_feature_cp[:, missing_sensor_id*6:(missing_sensor_id+1)*6] = 0
        return one_feature_cp
        
    def __len__(self):
        if self.mode == self.TRAIN_MODE:
            # although the transform generate more data, the index here is irrelevant
            return len(self.features)
        else:
            # take all available missing pattern * data number
            return len(self.features) * len(self.missing_index_list)
    
    def __getitem__(self, idx):
        if self.mode == self.TRAIN_MODE:
            missing_sensor_id_list = random.choice(self.missing_index_list)
            x = self.transform(self.features[idx], missing_sensor_id_list)
            label = self.label[idx]

        else:
            # take all available missing pattern
            missing_sensor_id_list = self.missing_index_list[ idx // len(self.features) ]
            x = self.transform(self.features[ idx % len(self.features) ], missing_sensor_id_list)
            label = self.label[idx % len(self.features)]
        return x, int(label)


In [7]:
class DataModule(pl.LightningDataModule):
    STANDARDIZE = False
    BATCH_SIZE = 512
    
    def __init__(self, test_user, missing_sensor_numbers):
        super().__init__()
        self.test_user = test_user
        self.missing_sensor_numbers = missing_sensor_numbers

    def load_data(self, mode):
        if mode == "train":
            folder_path = TRAIN_FOLDER_PATH
            data_file_name = train_data_file_name_
            label_file_name = train_label_file_name_

            train_val_data_list = None
            train_val_label_list = None
            
            for user in range(USER_NUM):
                if user == self.test_user: 
                    continue
                
                print("train_user", user)
                train_data_file_name = data_file_name.format(user)
                train_label_file_name = label_file_name.format(user)

                train_data_file_path = os.path.join(folder_path, train_data_file_name)
                train_label_file_path = os.path.join(folder_path, train_label_file_name)
                
                train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
                l, s, d, w = train_val_data.shape
                train_val_data = train_val_data.reshape(l, s * d, w).transpose(0,2,1)

                if train_val_data_list is None:
                    train_val_data_list = train_val_data
                    train_val_label_list = train_val_label
                else:
                    train_val_data_list = np.vstack((train_val_data_list, train_val_data))
                    train_val_label_list = np.concatenate((train_val_label_list, train_val_label))

            train_val_data = train_val_data_list
            train_val_label = train_val_label_list
            
        elif mode == "test":
            folder_path = TEST_FOLDER_PATH
            data_file_name = test_data_file_name_
            label_file_name = test_label_file_name_
    
            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            train_val_data = train_val_data.reshape(l, s * d, w).transpose(0,2,1)
    
        return train_val_data, train_val_label

    def setup(self, stage: str):
        # Assign Train/val split(s) for use in Dataloaders
        
        if stage == "validate" or stage == "fit":
            train_val_data, train_val_label = self.load_data("train")
            self.train_data, self.val_data, self.train_label, self.val_label = train_test_split(
                train_val_data, train_val_label, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

            self.train_dataset = CustomTrainDataset(
                CustomTrainDataset.TRAIN_MODE, self.train_data, self.train_label, missing_sensor_numbers=self.missing_sensor_numbers)
            self.val_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.val_data, self.val_label, missing_sensor_numbers=self.missing_sensor_numbers)

        elif stage == "test" :
            train_val_data, train_val_label = self.load_data("test")
            self.test_data = train_val_data
            self.test_label = train_val_label

            self.test_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.test_data, self.test_label, missing_sensor_numbers=self.missing_sensor_numbers)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.BATCH_SIZE, num_workers=0, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.BATCH_SIZE,  num_workers=0, shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.BATCH_SIZE,  num_workers=0, shuffle=False)

    def teardown(self, stage):
        print("teardown")
        if stage == "validate" or stage == "fit":
            del self.train_data, self.train_label
            del self.val_data, self.val_label
            del self.train_dataset
            del self.val_dataset
            
        elif stage == "test" :
            del self.test_data, self.test_label
            del self.test_dataset
        gc.collect()


In [9]:
data_module = DataModule(1, 1)
data_module.setup("fit")
data_module.train_data.shape

train_user 0
train_user 2
train_user 3
train_user 4
train_user 5
train_user 6
train_user 7
train_user 8
train_user 9
train_user 10
train_user 11
train_user 12
train_user 13
train_user 14
train_user 15


(73884, 256, 42)

In [10]:
data = data_module.train_dataset[0]
data[1], data[0][0]

(4,
 array([0.32876712, 0.36813223, 0.38270855, 0.23328531, 0.21495754,
        0.22658242, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.31556886, 0.39839226, 0.34658355,
        0.25972992, 0.24153638, 0.19600333, 0.4280699 , 0.34839636,
        0.33220407, 0.22292672, 0.22038516, 0.2235621 , 0.4224018 ,
        0.35157084, 0.30766958, 0.22106978, 0.21923521, 0.2231236 ,
        0.4319908 , 0.3506275 , 0.3310721 , 0.22761604, 0.22643924,
        0.2203628 , 0.4195144 , 0.34043145, 0.3387335 , 0.25530905,
        0.23053345, 0.23562102], dtype=float32))

In [11]:
del data_module, data
gc.collect()

1399

## MODEL DEFINITION

In [12]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        return weighted

In [13]:

class LSTMModel(pl.LightningModule):
    def __init__(self, hidden_size=128, input_size=42, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, 256, input_size)
        
        self.rnn = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=3,
                          batch_first=True)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)

        self.all_test = []
        self.all_pred = []
        
    def forward(self, x):
        activation, _ = self.rnn(x)
        
        b, _, _ = activation.size()
        lstm_output = activation[:,-1,:].view(b,-1)
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)

        self.all_pred = test_pred_labels
        self.all_test = y

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss)

In [14]:
class LSTMBiModel(pl.LightningModule):
    def __init__(self, hidden_size=128, input_size=42, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, 256, input_size)
        
        self.rnn = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=3,
                          batch_first=True,
                          bidirectional=True)

        double_hidden_size = hidden_size * 2
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * double_hidden_size, out_features=output_size)

        self.all_test = []
        self.all_pred = []
        
    def forward(self, x):
        activation, (h, _) = self.rnn(x)
        b, _, _ = activation.size()
        
        lstm_output = h[-2:].permute(1,0,2).reshape(b, -1)
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)

        self.all_pred = test_pred_labels
        self.all_test = y

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss)

In [15]:
class LSTMAttentionModel(pl.LightningModule):
    def __init__(self, hidden_size=128, input_size=42, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, 256, input_size)
        
        self.rnn1 = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention1 = SelfAttention(
            input_dim=hidden_size)

        self.rnn2 = nn.LSTM(input_size=hidden_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention2 = SelfAttention(
            input_dim=hidden_size
        )

        self.rnn3 = nn.LSTM(input_size=hidden_size, 
                  hidden_size=hidden_size,
                  num_layers=1,
                  batch_first=True)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
        self.all_test = []
        self.all_pred = []
        
    def forward(self, x):
        activation, _ = self.rnn1(x)
        activation = self.attention1(activation)
        activation, _ = self.rnn2(activation)
        activation = self.attention2(activation)
        activation, (h, _) = self.rnn3(activation)

        b, _, _ = activation.size()
        
        lstm_output = activation[:,-1,:].view(b,-1)
        
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
        y = y
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)
        
        self.all_pred = test_pred_labels
        self.all_test = y
        
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        y = y
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)
        

In [16]:

class CNNLSTMModel(pl.LightningModule):
    def __init__(self, hidden_size=128, sequence_length=256, input_size=42, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, sequence_length, input_size)
        
        self.cnn = nn.Conv1d(sequence_length, cnn_filter_size, kernel_size=5, padding="same")
        
        self.rnn = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=3,
                          batch_first=True)
 
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)

        self.all_test = []
        self.all_pred = []
        
    def forward(self, x):
        output = self.cnn(x)
        activation, _ = self.rnn(output)
        
        b, _, _ = activation.size()
        lstm_output = activation[:,-1,:].view(b,-1)
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)

        self.all_pred = test_pred_labels
        self.all_test = y

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss)

In [17]:
class CNNLSTMAttentionModel(pl.LightningModule):
    def __init__(self, hidden_size=128, sequence_length=256, input_size=42, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, sequence_length, input_size)
        
        self.cnn = nn.Conv1d(sequence_length, cnn_filter_size, kernel_size=5, padding="same")
        
        self.rnn1 = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention1 = SelfAttention(
            input_dim=hidden_size)

        self.rnn2 = nn.LSTM(input_size=hidden_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention2 = SelfAttention(
            input_dim=hidden_size
        )

        self.rnn3 = nn.LSTM(input_size=hidden_size, 
                  hidden_size=hidden_size,
                  num_layers=1,
                  batch_first=True)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
        self.all_test = []
        self.all_pred = []
        
    def forward(self, x):
        output = self.cnn(x)
        activation, _ = self.rnn1(output)
        activation = self.attention1(activation)
        activation, _ = self.rnn2(activation)
        activation = self.attention2(activation)
        activation, _ = self.rnn3(activation)

        b, _, _ = activation.size()
        
        lstm_output = activation[:,-1,:].view(b,-1)
        
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
        y = y
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)
        
        self.all_pred = test_pred_labels
        self.all_test = y
        
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        y = y
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)
        

In [18]:
train_model_class = [CNNLSTMModel, CNNLSTMAttentionModel, LSTMBiModel, LSTMAttentionModel, LSTMModel]


In [21]:
# only for test

def test():
    patience = 20
    missing_sensor_numbers = 4 # no missing sensor
    user = 2 # use user2 to test
    for model_class in train_model_class:
        data_module = DataModule(test_user=user, missing_sensor_numbers=missing_sensor_numbers)
        
        model = model_class(input_size=42, output_size=10)
        
        model_name = model.__class__.__name__
        print("Running for model", model_name)
        print("summary(model)", summary(model))

        tb_logger = TensorBoardLogger(f"./loggers/{model_name}")
        
        trainer = pl.Trainer(
            logger=tb_logger,
            callbacks=[EarlyStopping(monitor="val_loss", patience=patience, mode="min")],
            fast_dev_run = True
        )
        trainer.fit(model, data_module)
        trainer.test(model, data_module)
# test()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Running for model LSTMModel
summary(model) =================================================================
Layer (type:depth-idx)                   Param #
LSTMModel                                --
├─LSTM: 1-1                              352,256
├─Sequential: 1-2                        --
│    └─Linear: 2-1                       16,512
│    └─BatchNorm1d: 2-2                  256
│    └─Dropout1d: 2-3                    --
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       16,512
│    └─BatchNorm1d: 2-6                  256
│    └─Dropout1d: 2-7                    --
│    └─ReLU: 2-8                         --
├─Sequential: 1-3                        --
│    └─Linear: 2-9                       16,512
│    └─BatchNorm1d: 2-10                 256
│    └─Dropout1d: 2-11                   --
│    └─ReLU: 2-12                        --
│    └─Linear: 2-13                      16,512
│    └─BatchNorm1d: 2-14                 256
│    └─Dropout1d: 2-15   

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params | In sizes        | Out sizes                                           
-------------------------------------------------------------------------------------------------------------------
0 | rnn        | LSTM       | 352 K  | [1024, 256, 42] | [[1024, 256, 128], [[3, 1024, 128], [3, 1024, 128]]]
1 | seq_1      | Sequential | 33.5 K | [1024, 128]     | [1024, 128]                                         
2 | seq_2      | Sequential | 33.5 K | [1024, 128]     | [1024, 128]                                         
3 | classifier | Linear     | 3.9 K  | [1024, 384]     | [1024, 10]                                          
-------------------------------------------------------------------------------------------------------------------
423 K     Trainable params
0         Non-trainable params
423 K     Total params
1.693     Total estimated model params size (MB)


Epoch 0: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.63it/s]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.83it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


...

Epoch 0: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.48it/s, train_loss=2.320]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|████████████████████████████████| 1/1 [00:00<00:00, 10.90it/s, train_loss=2.320, val_loss=2.300]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|████████████████████████████████| 1/1 [00:00<00:00, 10.81it/s, train_loss=2.320, val_loss=2.300]
teardown


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 58.32it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                0.275390625
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
teardown


In [ ]:
batch_size = 512
patience = 20
# missing_sensor_numbers = 0
# all_test_pred = {}

missing_sensor_numbers = 0 ## Changed for no missing sensor

for missing_sensor_numbers in [1]:
    for user in range(USER_NUM): ## Changed for user 2 only
        for model_class in train_model_class: 
            start_timer = time.perf_counter()
            
            all_test = []
            all_pred = []
            print(f"\n*************training on User{user}*************")
            
            data_module = DataModule(test_user=user, missing_sensor_numbers=missing_sensor_numbers)
            
            model = model_class(input_size=42, output_size=10)
            model_name = model.__class__.__name__
            print("Running for model", model_name)
            
            tb_logger = TensorBoardLogger(f"./logger96_{missing_sensor_numbers}missing/{model_name}")
            
            trainer = pl.Trainer(
                logger=tb_logger,
                callbacks=[EarlyStopping(monitor="val_loss", patience=patience, mode="min")],
            )
        
            trainer.fit(model, data_module)
            trainer.test(model, data_module)
    
            all_test.extend(model.all_test)
            all_pred.extend(model.all_pred)
    
            test_target_pred = (all_test, all_pred)
            
            end_timer = time.perf_counter()
            exec_time = end_timer - start_timer

            save_dir = os.path.join(RESULT_FOLDER_PATH, f"96_{missing_sensor_numbers}missing")
            os.makedirs(save_dir, exist_ok=True)
            with open(os.path.join(save_dir, f"test_pred_user_{user}_{missing_sensor_numbers}_{model_name}_{exec_time:.0f}s.pkl"), "wb") as f:
                pickle.dump(test_target_pred, f)



## Calculate the performance of the model

In [ ]:
# missing_sensor_numbers = 0
# model_name = train_model_class[3].__name__
# print(model_name)

# all_test_pred = {}
# for user in range(USER_NUM):
#     with open(os.path.join(RESULT_FOLDER_PATH, "96", f"test_pred_user_{user}_{missing_sensor_numbers}_{model_name}.pkl"), "rb") as f:
#         all_test_pred[user] = pickle.load(f)

In [ ]:
# all_test_pred_result = ([], [])
# for user in range(USER_NUM):
#     all_test_pred_result[0].extend(all_test_pred[user][0])
#     all_test_pred_result[1].extend(all_test_pred[user][1])

# print(len(all_test_pred_result[0]))

In [ ]:
# all_test, all_pred = all_test_pred_result

# # print("missing index", missing_index)
# all_test_with_label = [label_list[i] for i in all_test]
# all_pred_with_label = [label_list[i] for i in all_pred]

# cf = confusion_matrix(all_test_with_label, all_pred_with_label, labels=label_list)
# sns.heatmap(cf, annot=True, xticklabels=eng_label_list, yticklabels=eng_label_list, fmt='g')

In [80]:
# from sklearn.metrics import accuracy_score, f1_score
# all_test = list(map(lambda x: x.cpu().item(), all_test))
# all_pred = list(map(lambda x: x.cpu().item(), all_pred))
# print("accuracy:", accuracy_score(all_test, all_pred))
# print("F1 score:", f1_score(all_test, all_pred, average='micro'))

accuracy: 1.0
F1 score: 1.0
